# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [4]:
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import of our own code
from inauguralproject import ExchangeEconomyClass

model = ExchangeEconomyClass()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Question 1

**Explain how you solve the model**

In [2]:
# code for solving the model (remember documentation and comments)

a = np.array([1,2,3])
b = inauguralproject.square(a)
print(b)

[1 4 9]


# Question 2

Explain your code and procedure

In [5]:
par = model.par

# Number of divisions
N = 20

# Calculate P1
P1 = [0.5 + 2 * i / N for i in range(N + 1)]

# Iterate over prices
for p1 in P1:
    # Calculate errors in market clearing condition
    eps1, eps2 = model.check_market_clearing(p1)
    
    # Print errors
    print(f"For p1 = {p1:.1f}, Error in market clearing for x1: {eps1:.2f}, Error in market clearing for x2: {eps2:.2f}")

For p1 = 0.5, Error in market clearing for x1: 0.53, Error in market clearing for x2: -0.27
For p1 = 0.6, Error in market clearing for x1: 0.34, Error in market clearing for x2: -0.21
For p1 = 0.7, Error in market clearing for x1: 0.21, Error in market clearing for x2: -0.15
For p1 = 0.8, Error in market clearing for x1: 0.11, Error in market clearing for x2: -0.09
For p1 = 0.9, Error in market clearing for x1: 0.03, Error in market clearing for x2: -0.03
For p1 = 1.0, Error in market clearing for x1: -0.03, Error in market clearing for x2: 0.03
For p1 = 1.1, Error in market clearing for x1: -0.08, Error in market clearing for x2: 0.09
For p1 = 1.2, Error in market clearing for x1: -0.13, Error in market clearing for x2: 0.15
For p1 = 1.3, Error in market clearing for x1: -0.16, Error in market clearing for x2: 0.21
For p1 = 1.4, Error in market clearing for x1: -0.20, Error in market clearing for x2: 0.27
For p1 = 1.5, Error in market clearing for x1: -0.22, Error in market clearing f

# Question 3

Explain your code and procedure

In [6]:
#IKKE RIGTIGT TROR JEG.
#Tror ikke det er rigtigt fordi error in market clearing for x1 og x2 ikke er 0 ved p1=2,5.

# Define tolerance
tolerance = 1e-6

# Iterate over prices
for price in P1:
    # Calculate errors in market clearing condition
    eps1, eps2 = model.check_market_clearing(price)
    
    # Check if errors are close to zero
    if abs(eps1) < tolerance and abs(eps2) < tolerance:
        # Market clearing price found
        price
        break

# Print the market clearing price
print(price)

2.5


In [8]:
#Måske er det rigtigt. Her får jeg også 2,5

# Iterate over prices
for price in P1:
    if eps1 == 0 and eps1 == 0:
        print("Market clearing price:", price)
print(price)

2.5


ADD CONCISE CONLUSION.